In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyer_loader import JupyerLoader

loader = JupyerLoader()
loader.load_all()

# AlphaBeta Pruning Engine

The previous example makes clear that the primary goal to increase the search depth must be a reduction of the nodes. The `MiniMax algorithm` is a depth-first search where the paths have to be evaluated completely one after the other. If it is the maximizing player's turn, i.e. white, the first evaluation of the path gives the minimum value white can reach. Each further path must exceed this value so that it is ultimately played by white. Similarly, for black on the move, each further path must undercut the previous value for it to be played by black. 

The `AlphaBeta Pruning Algorithm` builds on this idea and introduces two additional parameters `alpha` and `beta` for the `_value` function. `alpha` is the value that the current position has at least and beta is the value that the position has at most. Therefore `alpha <= _value(board, depth, alpha, beta) <= beta` holds.

First, a new class `AlphaBetaEngine` is defined again, which inherits from `MiniMaxEngine`.

In [ ]:
from converted_notebooks.s09_minimax_engine import MiniMaxEngine


class AlphaBetaEngine(MiniMaxEngine):
    pass

The `_value` function has the same interface as before except for the two new parameters `alpha` and `beta`. The termination conditions of the recursive function are also the same. 

For the maximizing player, hence the white player, `alpha` stores the current best score that can be achieved in that position. For each child node, alpha is therefore incremented with the statement `alpha = max(alpha, value)` if its value is higher. However, if `alpha` or the score of a child node is greater than `beta`, the search can be terminated prematurely. This is called a `beta cutoff`. The reason for this is that with `beta` Black could already achieve a better score for himself in another position and thus will not let White get into the current situation at all. Thus it is of no use to White to analyze the further moves. 

The same applies to the minimizing player. His currently best score is stored in `beta` and always lowered with the instruction `beta = min(beta, value)` if a child node has a low and therefore better value for black. If `beta` or the value of the child node is below `alpha`, the search can be aborted. This is called an `alpha cutoff`. Here, too, White had already found a better move for himself one level higher with `alpha` and will therefore not let Black get into this position. 

In [ ]:
import chess


def _value(self, board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    if score := self.evaluator.evaluate(board):
        return self.PLAYER_MULTIPLIER[board.turn] * score
    if depth == 0:
        return self.PLAYER_MULTIPLIER[board.turn
                                      ] * self.evaluator.heuristic(board)

    if board.turn is chess.WHITE:
        for move in board.legal_moves:
            board.push(move)
            value = self._value(board, depth - 1, alpha, beta)
            board.pop()
            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha
    else:
        for move in board.legal_moves:
            board.push(move)
            value = self._value(board, depth - 1, alpha, beta)
            board.pop()
            if alpha >= value:
                return value
            beta = min(beta, value)

        return beta


AlphaBetaEngine._value = _value

Finally, the `_evaluate_move` method must be adapted. This is identical to the implementation of the `MiniMaxEngine` with exception of the `_value` call. There you have to pass an initial value for `alpha` and `beta`. Since no move has been played yet, White can in any case achieve the worst possible result, he is set to mate. Accordingly `alpha` is initialized with `-self.VALUE_CHECKMATE`. The best possible result for white is to set mate himself, accordingly `beta` is initialized with `self.VALUE_CHECKMATE.

In [ ]:
from converted_notebooks.s04_engine_interface import ScoredMove


def _evaluate_move(self, board: chess.Board, move: chess.Move):
    board.push(move)
    score = self._value(
        board,
        self.look_ahead_depth,
        -self.evaluator.value_checkmate,
        self.evaluator.value_checkmate
    )
    board.pop()
    return ScoredMove(score=score, move=move)


AlphaBetaEngine._evaluate_move = _evaluate_move

Now the `AlphaBetaEngine` can evaluate the same constructed position `sample_minimax_board`. You can see in the graph that some nodes are marked with a question mark. These are the nodes that in this case did not have to be evaluated, because the result of the parent node was already defined before due to alpha and beta cutoffs.

In [ ]:
import random
import utils.min_max_tree
from converted_notebooks.s08_evaluation import standard_evaluator
from converted_notebooks.s09_minimax_engine import sample_minimax_board

random.seed(42)

engine = AlphaBetaEngine(evaluator=standard_evaluator, look_ahead_depth=2)
# engine = AlphaBetaEngine(evaluator=standard_evaluator, look_ahead_depth=3)
tree = utils.min_max_tree.add_tree_to_engine(engine)
print(engine.analyse(sample_minimax_board))
tree.draw()

For the second position 'middlegame_board', the number of nodes can again be calculated for depths two and three. At a depth of two, with 7'459 only about 10% of the previous nodes have to be examined, 62'660 paths were pruned. At a depth of three, with 10,7628 nodes only about 5% of the previous nodes have to be evaluated.

In [ ]:
import random
import utils.min_max_tree
from converted_notebooks.s08_evaluation import standard_evaluator
from converted_notebooks.s09_minimax_engine import middlegame_board, result_minimax

random.seed(42)

engine = AlphaBetaEngine(evaluator=standard_evaluator, look_ahead_depth=2)
# engine = AlphaBetaEngine(evaluator=standard_evaluator, look_ahead_depth=3)
tree = utils.min_max_tree.add_tree_to_engine(engine)
result_alphabeta = engine.analyse(middlegame_board)
tree.count()

assert result_minimax == result_alphabeta

From the operation of the algorithm, it is obvious that it is advantageous if the best path is evaluated first. In the best case, it is possible that the `x` nodes considered at `MiniMax` decrease to `sqrt(x)` when [Alpha Beta Pruning](https://www.chessprogramming.org/Alpha-Beta) is applied. If, on the other hand, the moves are sorted the other way around, so that the worst path is started with, no pruning is possible at all.

## NegaMax Algorithmus 

The `NegaMax algorithm` does not lead to a direct improvement of the engine, but simplifies the code of the `MiniMax algorithm` and all algorithms based on it. The basic idea here is that the evaluation of each node is done from the perspective of the player on the move. Thus both players are maximizing and the previous case distinction is no longer necessary. 

For the previous implementation of `_value` for the `AlphaBeta Pruning Algorithm`, this first of all changes the return values of the termination conditions. Since the value is now from the perspective of the player on the move and the functions `_evaluate` and `_absolute_heuristic` also evaluate from the perspective of the player on the move, the value can be returned directly. 
Furthermore, the case distinction is completely omitted and only the code for the maximizing player is needed. There the recursive call of the `_value` function changes to `value = -1 * self._value(board, depth - 1, -beta, -alpha)`. The return value, as well as alpha and beta, are negated to be from the player's perspective on the move. Additionally, alpha and beta must be swapped. 

In [ ]:
def _value(self, board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    if score := self.evaluator.evaluate(board):
        return score
    if depth == 0:
        return self.evaluator.heuristic(board)

    for move in board.legal_moves:
        board.push(move)
        value = -1 * self._value(board, depth - 1, -beta, -alpha)
        board.pop()
        if value >= beta:
            return value
        alpha = max(alpha, value)

    return alpha


AlphaBetaEngine._value = _value

The `_evaluate_move` needs to be adjusted so that its returned result is again from the perspective of white as before.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move):
    board.push(move)
    score = self._value(
        board,
        self.look_ahead_depth,
        -self.evaluator.value_checkmate,
        self.evaluator.value_checkmate
    )
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()
    return ScoredMove(score=score, move=move)


AlphaBetaEngine._evaluate_move = _evaluate_move

Now the `middlegame_board` position can also be analyzed by the `NegaMax` variant and afterwards it can be verified that all three implementations actually evaluate the position in the same way.

In [ ]:
random.seed(42)

engine = AlphaBetaEngine(evaluator=standard_evaluator, look_ahead_depth=2)
# engine = AlphaBetaEngine(evaluator=standard_evaluator, look_ahead_depth=3)
tree = utils.min_max_tree.add_tree_to_engine(engine)
result_negamax = engine.analyse(middlegame_board)
tree.count()

assert result_alphabeta == result_negamax